<a href="https://colab.research.google.com/github/zhannatoleubek-png/special-okx-chainsaw/blob/main/%D0%BB%D1%83%D1%87%D1%88%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =====================================================
# 🚀 OKX Impulse Scanner + Vumanchu Cipher B Logic + BTC Dominance fallback
# =====================================================
# Colab-ready: pip deps
!pip install --quiet ccxt pandas numpy requests ta

import time
import datetime as dt
import requests
import pandas as pd
import numpy as np
import ccxt
from ta.momentum import RSIIndicator
from ta.volume import OnBalanceVolumeIndicator
from ta.trend import EMAIndicator

# ---------------- CONFIG ----------------
CONFIG = {
    "telegram_bot_token": "8288179092:AAGczGGxYbHc0SqDrKO_BSayAZv1PL7ufJU",
    "telegram_chat_id": "381202205",
    "symbols": [],                  # пустой список = динамический подбор всех USDT
    "scan_interval": 10 * 60,       # 10 минут
    "atr_period": 14,
    "min_confidence_score": 6.0,
    "min_base_volume_24h": 100000
}

exchange = ccxt.okx({"enableRateLimit": True})

# ---------------- HELPERS ----------------
def send_telegram_message(bot_token, chat_id, text, parse_mode="HTML"):
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    payload = {"chat_id": chat_id, "text": text, "parse_mode": parse_mode, "disable_web_page_preview": True}
    try:
        r = requests.post(url, json=payload, timeout=10)
        print("📨 Telegram response:", r.status_code)
        return r.json()
    except Exception as e:
        print("❌ Telegram send error:", e)
        return None

def now_ts():
    return dt.datetime.now(dt.timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")

def safe_fetch_ohlcv(symbol, timeframe='1h', limit=200):
    try:
        return exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    except Exception as e:
        # debug print only, don't raise
        print(f"⚠ safe_fetch_ohlcv error {symbol} {timeframe}: {e}")
        return None

# ================= BTC DOMINANCE (OKX -> fallback CMC) =================
def get_btc_dominance_okx():
    """
    Попытка получить BTC Dominance с OKX (если OKX предоставляет).
    Возвращает float (проценты), либо None.
    """
    try:
        # OKX может иметь несколько публичных/непубличных эндпойнтов — пробуем простой запрос,
        # если структура иная или пустая — вернём None
        url = "https://www.okx.com/api/v5/market/index-components?indexName=BTC.D"
        r = requests.get(url, timeout=8)
        data = r.json()
        # ожидаем data["data"][0]["ratio"] или что-то похожее
        arr = data.get("data") or []
        if not arr:
            raise ValueError("OKX returned empty data")
        # пробуем извлечь поле ratio или dominance
        first = arr[0] if isinstance(arr, list) else arr
        dom = first.get("ratio") or first.get("dominance") or first.get("dominanceRatio")
        if dom is None:
            # иногда OKX отдаёт компонентную структуру; попробуем искать значение по ключам
            for item in arr:
                if isinstance(item, dict):
                    for k in ("ratio", "dominance", "dominanceRatio"):
                        if k in item and item[k] is not None:
                            dom = item[k]; break
                    if dom is not None:
                        break
        if dom is None:
            raise ValueError("OKX response missing dominance")
        return float(dom)
    except Exception as e:
        print("⚠️ BTC.D fetch error (OKX):", e)
        return None

def get_btc_dominance_cmc():
    """
    Фоллбек: пробуем CoinMarketCap public data-api (без API ключа — если доступен).
    Возвращаем float или None.
    """
    try:
        url = "https://api.coinmarketcap.com/data-api/v3/global-metrics/quotes/latest"
        r = requests.get(url, timeout=8)
        data = r.json()
        # структура: data["data"]["btcDominance"]
        d = data.get("data", {})
        btc_dom = d.get("btcDominance") or d.get("btc_dominance") or d.get("dominance", {}).get("btc")
        if btc_dom is None:
            # иногда структура другая:
            if "btcDominance" in d:
                btc_dom = d["btcDominance"]
        if btc_dom is None:
            raise ValueError("CMC response missing btcDominance")
        return float(btc_dom)
    except Exception as e:
        print("⚠️ BTC.D fetch error (CMC):", e)
        return None

def get_btc_dominance():
    """
    Сначала OKX, затем CoinMarketCap. Если оба не доступны — возвращаем None.
    """
    dom = get_btc_dominance_okx()
    if dom is not None:
        return dom
    dom = get_btc_dominance_cmc()
    return dom

# ================= INDICATORS =================
def compute_vwap(df):
    typical_price = (df['high'] + df['low'] + df['close']) / 3
    pv = typical_price * df['vol']
    vol_cum = df['vol'].cumsum().replace(0, np.nan)
    vwap = pv.cumsum() / vol_cum
    # fill initial nans safely
    vwap = vwap.fillna((pv.cumsum() / (df['vol'].cumsum().replace(0, np.nan))).bfill().ffill())
    return vwap

def money_flow_index(high, low, close, volume, window=14):
    typical_price = (high + low + close) / 3
    money_flow = typical_price * volume
    positive_flow = [0]
    negative_flow = [0]
    for i in range(1, len(typical_price)):
        if typical_price[i] > typical_price[i - 1]:
            positive_flow.append(money_flow[i]); negative_flow.append(0)
        elif typical_price[i] < typical_price[i - 1]:
            positive_flow.append(0); negative_flow.append(money_flow[i])
        else:
            positive_flow.append(0); negative_flow.append(0)
    pos_mf = pd.Series(positive_flow).rolling(window=window).sum()
    neg_mf = pd.Series(negative_flow).rolling(window=window).sum()
    denom = pos_mf + neg_mf
    mfi = 100 * (pos_mf / denom.replace(0, np.nan))
    return mfi.fillna(50.0)

def wavetrend(df, chlen=10, avg=21):
    df = df.copy()
    df['hlc3'] = (df['high'] + df['low'] + df['close']) / 3
    df['esa'] = df['hlc3'].ewm(span=chlen, adjust=False).mean()
    df['d'] = abs(df['hlc3'] - df['esa']).ewm(span=chlen, adjust=False).mean()
    df['ci'] = (df['hlc3'] - df['esa']) / (0.015 * df['d'].replace(0, np.nan))
    df['tci'] = df['ci'].ewm(span=avg, adjust=False).mean()
    df['wt1'] = df['tci']
    df['wt2'] = df['wt1'].ewm(span=4, adjust=False).mean()
    df['wt1'] = df['wt1'].fillna(0.0)
    df['wt2'] = df['wt2'].fillna(0.0)
    return df

# ================= BTC MULTI-TF SENTIMENT =================
def get_btc_sentiment():
    """
    Возвращает:
      - summary_str: текстовый краткий отчёт по BTC
      - primary_sent: 'bull'/'bear'/'neutral'
      - short_sent: 'bull'/'bear'/'neutral'
    Подробный таймфреймовый отчёт включён в summary_str.
    """
    symbol = 'BTC/USDT'
    tf_primary = ['1d', '12h', '4h', '1h', '30m']
    tf_short = ['15m', '5m', '3m', '1m']

    def tf_dir(tf):
        o = safe_fetch_ohlcv(symbol, tf, limit=220)
        if not o:
            return None
        df = pd.DataFrame(o, columns=['ts', 'open', 'high', 'low', 'close', 'vol'])
        df['close'] = df['close'].astype(float)
        try:
            ema50 = EMAIndicator(df['close'], window=50).ema_indicator().iloc[-1]
            ema200 = EMAIndicator(df['close'], window=200).ema_indicator().iloc[-1]
            price = df['close'].iloc[-1]
            if price > ema50 and price > ema200:
                return 'bull'
            elif price < ema50 and price < ema200:
                return 'bear'
            else:
                return 'neutral'
        except Exception:
            return None

    prim_dirs = [tf_dir(tf) for tf in tf_primary]
    short_dirs = [tf_dir(tf) for tf in tf_short]

    def majority(dir_list):
        cnt = {'bull': 0, 'bear': 0, 'neutral': 0}
        for x in dir_list:
            if x in cnt:
                cnt[x] += 1
        best = max(cnt.items(), key=lambda x: x[1])
        if best[1] == 0:
            return 'neutral'
        vals = list(cnt.values())
        if vals.count(best[1]) > 1:
            return 'neutral'
        return best[0]

    primary_sent = majority([d for d in prim_dirs if d is not None])
    short_sent = majority([d for d in short_dirs if d is not None])

    # build summary text
    prim_report = ", ".join([f"{tf}:{(d or '?')}" for tf, d in zip(tf_primary, prim_dirs)])
    short_report = ", ".join([f"{tf}:{(d or '?')}" for tf, d in zip(tf_short, short_dirs)])

    btc_dom = get_btc_dominance()
    dom_change = "—"
    if hasattr(get_btc_sentiment, "prev_dom") and btc_dom is not None:
        try:
            dom_change = round(btc_dom - get_btc_sentiment.prev_dom, 2)
        except:
            dom_change = "—"
    get_btc_sentiment.prev_dom = btc_dom or 0

    dom_str = f"{btc_dom:.2f}%" if btc_dom is not None else "n/a"
    change_str = f"{'+' if isinstance(dom_change, (int, float)) and dom_change>=0 else ''}{dom_change}" if dom_change != "—" else dom_change

    # influence short phrase
    influence = ""
    if btc_dom is None:
        influence = "⚠️ BTC Dominance data unavailable"
    else:
        if primary_sent == "bull" and isinstance(dom_change, (int, float)) and dom_change >= 0:
            influence = "🟢 BTC dominance rising — stronger BTC influence (supports LONGs)"
        elif primary_sent == "bear" and isinstance(dom_change, (int, float)) and dom_change >= 0:
            influence = "🔴 BTC dominance rising — BTC-driven flows (supports SHORTs on alts)"
        elif isinstance(dom_change, (int, float)) and dom_change < 0:
            influence = "🟡 BTC dominance falling — capital into alts (alt-season potential)"

    summary = f"BTC Trend — Primary({prim_report}) | Short({short_report})\nPrimary sentiment: {primary_sent.upper()}, Short sentiment: {short_sent.upper()}\nDominance: {dom_str} ({change_str})\n{influence}"
    return summary, primary_sent, short_sent

# ================= VUMANCHU CIPHER ANALYZER (LONG+SHORT) =================
def analyze_vumanchu_cipher(symbol):
    # BTC sentiment first
    btc_summary, btc_primary_sent, btc_short_sent = get_btc_sentiment()

    # load symbol ohlcv
    ohlcv_1h = safe_fetch_ohlcv(symbol, '1h', 200)
    ohlcv_15m = safe_fetch_ohlcv(symbol, '15m', 200)
    ohlcv_3m = safe_fetch_ohlcv(symbol, '3m', 300)
    if not ohlcv_1h or not ohlcv_15m or not ohlcv_3m:
        return None

    df1h = pd.DataFrame(ohlcv_1h, columns=['ts', 'open', 'high', 'low', 'close', 'vol'])
    df15 = pd.DataFrame(ohlcv_15m, columns=['ts', 'open', 'high', 'low', 'close', 'vol'])
    df3 = pd.DataFrame(ohlcv_3m, columns=['ts', 'open', 'high', 'low', 'close', 'vol'])
    for df in [df1h, df15, df3]:
        df['close'] = df['close'].astype(float)
        df['high'] = df['high'].astype(float)
        df['low'] = df['low'].astype(float)
        df['vol'] = df['vol'].astype(float)

    # 1H trend for asset
    ema50_1h = EMAIndicator(df1h['close'], 50).ema_indicator()
    ema200_1h = EMAIndicator(df1h['close'], 200).ema_indicator()
    trend_1h_bull = df1h['close'].iloc[-1] > ema50_1h.iloc[-1] and df1h['close'].iloc[-1] > ema200_1h.iloc[-1]
    trend_1h_bear = df1h['close'].iloc[-1] < ema50_1h.iloc[-1] and df1h['close'].iloc[-1] < ema200_1h.iloc[-1]

    # indicators on 3m
    df3 = wavetrend(df3)
    df3['ema8'] = EMAIndicator(df3['close'], 8).ema_indicator()
    df3['ema21'] = EMAIndicator(df3['close'], 21).ema_indicator()
    df3['ema50'] = EMAIndicator(df3['close'], 50).ema_indicator()
    df3['rsi'] = RSIIndicator(df3['close'], 14).rsi()
    df3['mfi'] = money_flow_index(df3['high'], df3['low'], df3['close'], df3['vol'], 14)
    df3['obv'] = OnBalanceVolumeIndicator(df3['close'], df3['vol']).on_balance_volume()
    df3['vwap'] = compute_vwap(df3)

    last = df3.iloc[-1]
    prev = df3.iloc[-2]

    # LONG conditions
    wt_cross_up = (prev['wt1'] < prev['wt2']) and (last['wt1'] > last['wt2'])
    ema_bull = (last['ema8'] > last['ema21']) and (last['close'] > last['ema8'])
    vwap_ok = (last['close'] >= last['vwap']) or (abs(last['close'] - last['vwap']) / last['vwap'] < 0.003)
    rsi_ok = last['rsi'] > 50 and last['rsi'] > prev['rsi']
    mfi_ok = last['mfi'] > 50 or (last['mfi'] - prev['mfi'] > 5)
    vol_mean = df3['vol'].rolling(20).mean().iloc[-1] if len(df3) >= 20 else df3['vol'].mean()
    vol_ok = (last['vol'] >= vol_mean * 1.1) or (last['obv'] > prev['obv'])
    long_conditions = [wt_cross_up, ema_bull, vwap_ok, rsi_ok, mfi_ok, vol_ok]
    long_score = sum(long_conditions)

    # SHORT conditions (inverted)
    wt_cross_down = (prev['wt1'] > prev['wt2']) and (last['wt1'] < last['wt2'])
    ema_bear = (last['ema8'] < last['ema21']) and (last['close'] < last['ema8'])
    vwap_ok_short = (last['close'] <= last['vwap']) or (abs(last['close'] - last['vwap']) / last['vwap'] < 0.003)
    rsi_ok_short = last['rsi'] < 50 and last['rsi'] < prev['rsi']
    mfi_ok_short = last['mfi'] < 50 or (prev['mfi'] - last['mfi'] > 5)
    vol_ok_short = (last['vol'] >= vol_mean * 1.1) or (last['obv'] < prev['obv'])
    short_conditions = [wt_cross_down, ema_bear, vwap_ok_short, rsi_ok_short, mfi_ok_short, vol_ok_short]
    short_score = sum(short_conditions)

    # Decide permitted signals by BTC short sentiment (short_sent)
    # If BTC short sentiment is 'bear' -> prefer/allow shorts; if 'bull' -> prefer/allow longs; 'neutral' allow both
    # Acquire btc_short_sent from get_btc_sentiment (already computed above)
    # Re-run to obtain the short sentiment as string
    # (we already have btc_summary string, but need short sentiment value too)
    # For simplicity call get_btc_sentiment again (cheap)
    _, btc_primary_sent, btc_short_sent = get_btc_sentiment()

    long_allowed = True
    short_allowed = True
    if btc_short_sent == 'bull':
        short_allowed = False
    elif btc_short_sent == 'bear':
        long_allowed = False

    # Do not go against asset 1H main trend
    if trend_1h_bull:
        short_allowed = False
    if trend_1h_bear:
        long_allowed = False

    signal_payloads = []

    # LONG branch
    if long_allowed and long_score >= 4:
        df3['low_roll'] = df3['low'].rolling(5).min()
        recent_low = float(df3['low_roll'].iloc[-1])
        sl = recent_low * 0.997
        risk = last['close'] - sl if last['close'] > sl else max(1e-8, last['close'] * 0.002)
        tp = last['close'] + 2 * risk
        signal_type = "STRONG LONG" if long_score == 6 else "LONG"

        msg = (
            f"{btc_summary}\n\n"
            f"🔔 <b>{symbol}</b> — {signal_type}\n"
            f"Entry: {last['close']:.6f}\nSL: {sl:.6f}\nTP: {tp:.6f}\n"
            f"Score: {long_score}/6 (WT, EMA ribbon, VWAP, RSI, MFI, Volume)\n"
            f"Indicators: RSI {last['rsi']:.1f}, MFI {last['mfi']:.1f}, WTΔ {last['wt1'] - last['wt2']:.3f}\n"
            f"{now_ts()}"
        )
        send_telegram_message(CONFIG['telegram_bot_token'], CONFIG['telegram_chat_id'], msg)
        signal_payloads.append({"side": "long", "score": long_score, "entry": last['close'], "SL": sl, "TP": tp,
                                "indicators": {"rsi": float(last['rsi']), "mfi": float(last['mfi']), "wt_delta": float(last['wt1'] - last['wt2'])}})

    # SHORT branch
    if short_allowed and short_score >= 4:
        df3['high_roll'] = df3['high'].rolling(5).max()
        recent_high = float(df3['high_roll'].iloc[-1])
        sl = recent_high * 1.003
        risk = sl - last['close'] if sl > last['close'] else max(1e-8, last['close'] * 0.002)
        tp = last['close'] - 2 * risk
        signal_type = "STRONG SHORT" if short_score == 6 else "SHORT"

        msg = (
            f"{btc_summary}\n\n"
            f"🔔 <b>{symbol}</b> — {signal_type}\n"
            f"Entry: {last['close']:.6f}\nSL: {sl:.6f}\nTP: {tp:.6f}\n"
            f"Score: {short_score}/6 (WT, EMA ribbon, VWAP, RSI, MFI, Volume) [SHORT]\n"
            f"Indicators: RSI {last['rsi']:.1f}, MFI {last['mfi']:.1f}, WTΔ {last['wt1'] - last['wt2']:.3f}\n"
            f"{now_ts()}"
        )
        send_telegram_message(CONFIG['telegram_bot_token'], CONFIG['telegram_chat_id'], msg)
        signal_payloads.append({"side": "short", "score": short_score, "entry": last['close'], "SL": sl, "TP": tp,
                                "indicators": {"rsi": float(last['rsi']), "mfi": float(last['mfi']), "wt_delta": float(last['wt1'] - last['wt2'])}})

    return signal_payloads if signal_payloads else None

# ================= SCAN WRAPPER =================
def scan_symbol_multitimeframe(sym, cfg):
    try:
        return analyze_vumanchu_cipher(sym)
    except Exception as e:
        print(f"❌ {sym} analysis error:", e)
        return None

# ================= MAIN LOOP =================
def main_loop(cfg):
    symbols = cfg['symbols'] or [s for s in exchange.load_markets() if 'USDT' in s]
    print(f"✅ Found {len(symbols)} filtered symbols")
    while True:
        print(f"{now_ts()} — scanning {len(symbols)} symbols...")
        for i, sym in enumerate(symbols, 1):
            try:
                print(f"[{i}/{len(symbols)}] Scanning {sym}...")
                res = scan_symbol_multitimeframe(sym, cfg)
                if res:
                    print(f" -> Signals for {sym}: {res}")
            except Exception as e:
                print(f"❌ Error scanning {sym}: {e}")
        print(f"{now_ts()} — waiting {cfg['scan_interval']} sec...\n")
        time.sleep(cfg['scan_interval'])

# ================= RUN =================
if __name__ == "__main__":
    main_loop(CONFIG)


✅ Found 559 filtered symbols
2025-10-21 09:01:46 UTC — scanning 559 symbols...
[1/559] Scanning USDT/SGD...
⚠️ BTC.D fetch error (OKX): OKX returned empty data
⚠️ BTC.D fetch error (OKX): OKX returned empty data
[2/559] Scanning USDT/AUD...
⚠️ BTC.D fetch error (OKX): OKX returned empty data
⚠️ BTC.D fetch error (OKX): OKX returned empty data
[3/559] Scanning USDT/AED...
⚠️ BTC.D fetch error (OKX): OKX returned empty data
⚠️ BTC.D fetch error (OKX): OKX returned empty data
[4/559] Scanning USDT/BRL...
⚠️ BTC.D fetch error (OKX): OKX returned empty data
⚠️ BTC.D fetch error (OKX): OKX returned empty data
[5/559] Scanning USDT/EUR...
⚠️ BTC.D fetch error (OKX): OKX returned empty data
⚠️ BTC.D fetch error (OKX): OKX returned empty data
[6/559] Scanning USDT/TRY...
⚠️ BTC.D fetch error (OKX): OKX returned empty data
⚠️ BTC.D fetch error (OKX): OKX returned empty data
📨 Telegram response: 200
 -> Signals for USDT/TRY: [{'side': 'short', 'score': np.int64(5), 'entry': np.float64(42.01), 'SL